In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import SplineGeometry, CSG2d , Rectangle
import matplotlib.pyplot as plt
import numpy as np

importing NGSolve-6.2.2204


In [2]:
from ngsolve.comp import pml

### CSG 2D 

In [3]:
geo = CSG2d ()
rect = Rectangle (( -6 , -20) ,(6 ,0) , bc='wall')
pmlregion = Rectangle (( -6 , -20) ,(6 ,-22) , bc='pmlregion')
hall = rect 
geo.Add(hall)
geo.Add(pmlregion)

mesh = Mesh(geo.GenerateMesh (maxh=0.2))

f = 5*exp(-10*((x-0.3)*(x-0.3)+(y+10)**2))
Draw(f, mesh, 'f')

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

BaseWebGuiScene

In [4]:
mesh.SetPML(pml.Cartesian(mins = [-6, -20], maxs = [6, -22],alpha=1j), "pmlregion")

### Spline Geometry Version

In [77]:


geo = SplineGeometry()
geo.AddRectangle (( -6 , -20) ,(6 ,0) , bc='wall')
geo.AddRectangle (( -6 , -20) ,(6 ,-22) , bc='pml')

geo.SetMaterial(1, "inner")
geo.SetMaterial(2, "pmlregion")

In [38]:
mesh = Mesh(geo.GenerateMesh (maxh=0.2))
mesh.Curve(3)

f = 5*exp(-10**2*((x-0.3)*(x-0.3)+(y+10)**2))
Draw(f, mesh, 'f')

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

BaseWebGuiScene

In [39]:
mesh.SetPML(pml.Cartesian(mins = [-6, -20], maxs = [6, -22],alpha=1j), "pmlregion")
mesh.GetBoundaries()

('wall',
 'wall',
 'wall',
 'wall',
 'pmlregion',
 'pmlregion',
 'pmlregion',
 'pmlregion')

### Solution

In [5]:
V = H1(mesh, order=4, complex=True)
u,v = V.TnT()

freq = 100
omega = 2*np.pi*freq
c0 = 333

a = BilinearForm(V)
a += c0**2*grad(u)*grad(v)*dx - omega**2*u*v*dx
a += -c0**2*1j*omega*u*v*ds("pmlregion")                         #PML

b = LinearForm(V)
b += f * v * dx

a.Assemble()
b.Assemble()

gfu = GridFunction(V)
gfu.vec.data = a.mat.Inverse() * b.vec
Draw(gfu)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

BaseWebGuiScene